In [0]:
--------------------- step 1 ------------------------
create temporary table billing_first_created
select 
min(created_at) as first_created_at,
min(website_pageview_id) as min_pv_id,
CAST( min(created_at) AS DATE) as first_day
from mavenfuzzyfactory.website_pageviews
where pageview_url = '/billing-2' and created_at is not null

-- 2012-09-10 -- 53550

--------------------- step 2 ------------------------
select 
  wp.website_session_id,
  wp.pageview_url as billing_version_seen,
  o.order_id
from website_pageviews wp
left join orders o
on 
  o.website_session_id = wp.website_session_id
where 
  wp.created_at <='2012-11-12'
  and wp.website_pageview_id >= 53550 
  and wp.pageview_url in ('/billing','/billing-2')
order by 
  wp.website_session_id,
  wp.created_at

--------------------- step 3 ------------------------
select 
  billing_version_seen,
  count(distinct website_session_id) as sessions,
  count(distinct order_id) as orders,
  count(distinct order_id)/count(distinct website_session_id) as billing_to_order_rate
from (
  select 
    wp.website_session_id,
    wp.pageview_url as billing_version_seen,
    o.order_id
  from website_pageviews wp
  left join orders o
  on 
    o.website_session_id = wp.website_session_id
  where 
    wp.created_at <='2012-11-12'
    and wp.website_pageview_id >= 53550 
    and wp.pageview_url in ('/billing','/billing-2')
) AS billing_sessions_w_orders
GROUP BY 1

